In [ ]:
import logging
import time
import random
import json
from datasets import load_dataset
import boto3
from botocore.config import Config
from botocore.exceptions import ClientError
from tqdm import tqdm

/Users/devanshk/Desktop/CSI5386-NLP/nlp/lib/python3.12/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [4]:
from dotenv import load_dotenv
load_dotenv()

True

In [5]:
# === Setup logging ===
logger = logging.getLogger(__name__)
logging.basicConfig(level=logging.INFO)

In [12]:
# === Constants ===
REQUEST_DELAY = 2  # Base delay between requests in seconds
MAX_RETRIES = 5    # Maximum number of retry attempts
BACKOFF_FACTOR = 2 # Exponential backoff factor
JITTER = 0.5       # Maximum jitter factor in seconds

# === Configure Bedrock client ===
custom_config = Config(
    connect_timeout=840,
    read_timeout=840,
    retries={'max_attempts': MAX_RETRIES}
)
bedrock_client = boto3.client(
    'bedrock-runtime',
    region_name='us-east-1',
    config=custom_config
)
# === Load Dataset ===
ds = load_dataset("openai/gsm8k", "main")
# === Function to handle exponential backoff with jitter ===
def exponential_backoff_with_jitter(attempt):
    """Calculate sleep time using exponential backoff with jitter."""
    backoff_time = REQUEST_DELAY * (BACKOFF_FACTOR ** attempt)
    sleep_time = backoff_time + random.uniform(0, JITTER)
    return sleep_time

# === Function to generate conversation with retries ===
def generate_conversation_with_retries(bedrock_client, model_id, system_prompts, messages):
    for attempt in range(MAX_RETRIES):
        try:
            response = bedrock_client.converse(
                modelId=model_id,
                messages=messages,
                system=system_prompts,
                inferenceConfig={"temperature": 0.7, "maxTokens": 4096},
            )
            return response
        except ClientError as e:
            error_code = e.response['Error']['Code']
            if error_code == 'ThrottlingException':
                sleep_time = exponential_backoff_with_jitter(attempt)
                logger.warning(f"ThrottlingException encountered. Retrying in {sleep_time:.2f} seconds...")
                time.sleep(sleep_time)
            else:
                logger.error(f"ClientError encountered: {e}")
                raise
    raise Exception("Max retries reached for throttling.")


# === Core Function ===
def generate_conversation(bedrock_client, model_id, system_prompts, messages):
    logger.info("Generating message with model %s", model_id)

    temperature = 0.5
    max_tokens = 4096

    inference_config = {
        "temperature": temperature,
        "maxTokens": max_tokens,
    }

    response = bedrock_client.converse(
        modelId=model_id,
        messages=messages,
        system=system_prompts,
        inferenceConfig=inference_config,
    )

    token_usage = response['usage']
    logger.info("Input tokens: %s", token_usage['inputTokens'])
    logger.info("Output tokens: %s", token_usage['outputTokens'])
    logger.info("Total tokens: %s", token_usage['totalTokens'])
    logger.info("Stop reason: %s", response['stopReason'])

    return response

# === Load Dataset ===
ds = load_dataset("openai/gsm8k", "main")

# === Configure Bedrock client ===
custom_config = Config(connect_timeout=840, read_timeout=840)
bedrock_client = boto3.client('bedrock-runtime', 
                            region_name='us-east-1',
                            config=custom_config)
   
# === System Prompt ===
system_prompts = [{
    "text": """You are a math teacher who writes step-by-step reasoning in Python code.

For each problem:
1. Break down the question into sub-steps using Python.
2. Use comments (#) to explain each step.
3. Do all calculations using variables.
4. Assign the final result to a variable named `answer`.

Make sure the final line is: `answer = ...` with the correct value.

Do not give any explanations outside of code. Just write Python code only.
Structure your response like this:

<code>:
<your Python code here>
</code>
Make sure your last line in the code is: answer = ...
"""
}]

# === Main Loop with Progress Saving ===
output_file = 'math_reasoning_dataset.jsonl'
start_index = 284


# Resume from existing file if available
try:
    with open(output_file, 'r') as f:
        lines = f.readlines()
        start_index = len(lines)
        logger.info(f"Resuming from index {start_index}")
except FileNotFoundError:
    pass

for i in tqdm(range(start_index, 350)):  # Adjust total number as needed
    example = ds["train"][i]

    message = {
        "role": "user",
        "content": [{"text": f"Given this math problem: {example['question']}\n\nProvide your reasoning step by step, then give the final answer."}]
    }

    messages = [message]

    try:
        response = generate_conversation_with_retries(
            bedrock_client,
            "us.deepseek.r1-v1:0",
            system_prompts,
            messages
        )

        reasoning_content = response['output']['message']['content'][1]['reasoningContent']
        reasoning_text = reasoning_content['reasoningText']['text']
        text_response = response['output']['message']['content'][0]['text']

        dataset_entry = {
            "instruction": example["question"],
            "output": f"<think>{reasoning_text}</think><answer>{text_response}</answer>",
            "original_answer": example["answer"]
        }

        # Save immediately after each successful example
        with open(output_file, 'a') as f:
            f.write(json.dumps(dataset_entry) + '\n')

    except Exception as e:
        logger.error(f"Error processing example {i}: {e}")
        continue

    # Rate limiting
    time.sleep(REQUEST_DELAY)

INFO:__main__:Resuming from index 283
100%|██████████| 67/67 [1:01:04<00:00, 54.70s/it]


In [ ]:
import pandas as pd
df = pd.read_jso    n("math_reasoning_dataset.jsonl", lines=True)

In [3]:
print(len(df))

350


In [113]:
df["output"][0]

"<think>Okay, let's see. Natalia sold clips in April and May. The problem says she sold to 48 friends in April. Then in May, she sold half as many as in April. We need to find the total for both months.\n\nFirst, let me note down the April sales: 48 clips. Then, May is half of April. So half of 48 is 24. Then add them together. 48 plus 24 equals 72. So the total is 72 clips.\n\nWait, but I need to write this in Python code. Let me structure it step by step. Assign April's number to a variable. Calculate May's as half of April. Then sum both. Make sure each step has comments. Use variables and then the answer variable at the end.\n</think><answer>\n\n<code>\n# Number of clips sold in April\napril_sales = 48\n# Calculate May sales as half of April's sales\nmay_sales = april_sales / 2\n# Total clips sold in April and May\ntotal_sales = april_sales + may_sales\nanswer = total_sales\n</code></answer>"

In [ ]:
"<think>Okay, let's see. Natalia sold clips to 48 friends in April. Then in May, she sold half as many as she did in April. We need to find the total for both months.\n\nFirst, the April sales are straightforward: 48 clips. Then for May, half of April's number. So half of 48 is 24. Then add them together: 48 plus 24 equals 72. So the total is 72 clips.\n</think><answer>\n\n<code>\n# Calculate the number of clips sold in April\napril_sales = 48\n\n# Calculate May's sales as half of April's sales\nmay_sales = april_sales / 2\n\n# Sum the sales from both months\ntotal_sales = april_sales + may_sales\n\nanswer = total_sales\n</code></answer>"

In [79]:
local_vars = {}
python_code = df["output"][0].split("<code>")[1].split("</code>")[0]
exec(python_code, {}, local_vars)

In [80]:
predicted_answer = local_vars.get("answer", None)

In [81]:
print(predicted_answer)  # Should print 72

72.0


In [89]:
ds["train"]["answer"][3]

'Maila read 12 x 2 = <<12*2=24>>24 pages today.\nSo she was able to read a total of 12 + 24 = <<12+24=36>>36 pages since yesterday.\nThere are 120 - 36 = <<120-36=84>>84 pages left to be read.\nSince she wants to read half of the remaining pages tomorrow, then she should read 84/2 = <<84/2=42>>42 pages.\n#### 42'

In [5]:
import pandas as pd
from datasets import load_dataset
ds = load_dataset("openai/gsm8k", "main")

/Users/devanshk/Desktop/CSI5386-NLP/nlp/lib/python3.12/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [17]:
print(ds["test"]["question"][5])
ds["test"]["answer"][5]

Kylar went to the store to buy glasses for his new apartment. One glass costs $5, but every second glass costs only 60% of the price. Kylar wants to buy 16 glasses. How much does he need to pay for them?


'The discount price of one glass is 60/100 * 5 = $<<60/100*5=3>>3.\nIf every second glass is cheaper, that means Kylar is going to buy 16 / 2 = <<16/2=8>>8 cheaper glasses.\nSo for the cheaper glasses, Kylar is going to pay 8 * 3 = $<<8*3=24>>24.\nAnd for the regular-priced glasses, Kylar will pay 8 * 5 = $<<8*5=40>>40.\nSo in total Kylar needs to pay 24 + 40 = $<<24+40=64>>64 for the glasses he wants to buy.\n#### 64'

In [ ]:
<think>
To solve this problem, we need to calculate the cost of the first glass and the cost of the second glass, and then add them together. Since the cost of every second glass is 60% of the price of the first glass, we need to find 60% of the cost of the first glass. We can do this by multiplying the cost of the first glass by 0.6.
</think>
<code>:
# Cost of the first glass
cost_first_glass = 5
# Calculate the cost of the second glass (60% of the cost of the first glass)
cost_second_glass = cost_first_glass * 0.6
# Calculate the total number of glasses Kylar wants to buy
total_glasses = 16
# Calculate the total cost of the glasses
total_cost = cost_first_glass + cost_second_glass + (cost_first_glass * (total_glasses - 2) * 0.6)
# Calculate the cost of the remaining glasses (every second glass is 60% of the price of the first glass)
remaining_glasses = total_glasses - 2
# Calculate the cost of the remaining glasses
remaining_cost = remaining_glasses * cost_first_glass * 0.6
# Calculate the total cost of all glasses
total_cost = cost_first_glass + cost_second_glass + remaining_cost
answer = total_cost
</code>

In [18]:
cost_first_glass = 5
# Calculate the cost of the second glass (60% of the cost of the first glass)
cost_second_glass = cost_first_glass * 0.6
# Calculate the total number of glasses Kylar wants to buy
total_glasses = 16
# Calculate the total cost of the glasses
total_cost = cost_first_glass + cost_second_glass + (cost_first_glass * (total_glasses - 2) * 0.6)
# Calculate the cost of the remaining glasses (every second glass is 60% of the price of the first glass)
remaining_glasses = total_glasses - 2
# Calculate the cost of the remaining glasses
remaining_cost = remaining_glasses * cost_first_glass * 0.6
# Calculate the total cost of all glasses
total_cost = cost_first_glass + cost_second_glass + remaining_cost
answer = total_cost

In [19]:
answer

50.0